In [27]:
# 爬蟲
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#斷詞所需套件
import jieba
# 爬蟲所需套件
import requests
from bs4 import BeautifulSoup
from googletrans import Translator
#GUI介面-tkinter所需套件
from tkinter import *
import tkinter as tk
#匯入Python Imaging Library 
from PIL import ImageTk, Image 
#檔案對話方塊所需套件
from tkinter import filedialog

In [28]:

def open_img(): 
    # 圖片檔案位置x
    global x 
    x = openfilename() 
    # 開啟圖片 
    img = Image.open(x) 
    # ANTIALIAS:平滑濾波，用於改變圖片尺寸  resize the image and apply a high-quality down sampling filter 
    img = img.resize((200, 200), Image.ANTIALIAS) 
    # PhotoImage:建立圖片物件，使用label顯示圖片 
    img = ImageTk.PhotoImage(img) 
    # 創建label 
    panel = Label(window, image = img,bg='white') 
    # set the image as img 
    panel.image = img 
    panel.pack(side='left',ipadx=10, padx=1)
    
def identify():
    file_path = x
    # read file with tf.io.read_file
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    # identify and get the class of fruit image
    image_class = model.predict_classes(tf.reshape(img, [1,64,64,3]))[0]
  #  global predict_name
    predict_name = class_name[image_class]   
    #進行斷詞
    list=jieba.lcut(predict_name)
    #將大寫變小寫 
    global target
    target = list[0].lower()
    # print the result of identification
    identify_label = tk.Label(window,text=predict_name,width=10,height=10,justify='left',bg='white')
    identify_label.pack(side='right',ipadx=10,padx=1)

    
import tkinter.ttk as ttk
import csv
# 讀入中文營養資訊 & 整理 & 輸出csv檔
def OutputCSV():   
    df = pd.read_html('https://docs.google.com/spreadsheets/d/e/2PACX-1vRACnRPosGeh5QPwLuU53UTOjbQf7uSZHnjDp2ecab8_AFE6yjaIxForntAtBahScFSCYlGh9mVhI-x/pubhtml',encoding='utf-8')
    t1 = df[0]
    # 調整表頭名稱
    t1.columns=['#','分析項分類','分析項','單位','每100克含量','名稱']
    t2 = t1.drop("#",axis = 1) #刪除1st行
    t3 = t2.drop([0],axis = 0) #刪除1st列
    # 篩選符合predict_name的蔬果名稱，產出新的df
    newdf = t3[t3.名稱.str.contains(target)]
    # 將df轉成csv檔
    Result = 'D:/nutri_result.csv'
    newdf.to_csv( Result )
    

OutputCSV()

In [29]:
def openfilename(): 
    # 開啟檔案對話方塊，並將檔案位置回傳
    filename = filedialog.askopenfilename(title ='Open') 
    return filename 

def printSomething():
    #爬取目標網址
    target_name = target.lower()
    url ='https://fruitsandveggies.org/fruits-and-veggies/'+target+'/'
    resp = requests.get(url)
    #若找不到相對網址則顯示'Could not find the relevant information'
    if (resp.status_code == 404):
        text = 'Sorry,we could not find the relevant information.'
    #若找到相對網址則進行爬蟲
    else:
        soup = BeautifulSoup(resp.text,'lxml')
        page = soup.find_all('div',class_='l-grid-item-medium-12')
        
        title = page[0].find_all('h4')
        content = page[0].find_all('p')
        for i in range(len(title)):
            if(i==2):
                text = '●'+title[i].get_text()+'\n'+content[2].get_text()+'\n'
            elif(i==3):
                text = '●'+title[i].get_text()+'\n'+content[2].get_text()+'\n'
                for i in range(len(content)):
                    if(i>2 and i<6):
                        if(i%2 == 1):
                            text = text + '●' + content[i].get_text() +'\n' 
                        else:                    
                            text = text + content[i].get_text() +'\n' 
                    elif(i>6):
                        if(i%2 == 0):
                            text = text + '●' + content[i].get_text() +'\n' 
                        else:
                            text = text + content[i].get_text() +'\n'

    # 將所得到之內容以google翻譯翻譯成中文
    pipi = Translator()
    result = pipi.translate(text,dest='zh-tw')
    result_label = tk.Label(window,text=result.text,width=50,height=50,wraplength=260,justify='left',bg='white')
    result_label.pack(side='right',ipadx=10,padx=1)

In [30]:
# 營養資訊表格視窗
def printNutriTable():
    root = Tk()
    root.title("Nutrition Table")
    width = 700
    height = 400
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width / 2) - (width / 2)
    y = (screen_height / 2) - (height / 2)
    root.geometry("%dx%d+%d+%d" % (width, height, x, y))
    root.resizable(0, 0)

    TableMargin = Frame(root, width=500)
    TableMargin.pack(side=TOP)

    scrollbarx = Scrollbar(TableMargin, orient=HORIZONTAL)
    scrollbary = Scrollbar(TableMargin, orient=VERTICAL)
    #設定表格尺寸與欄位名稱
    tree = ttk.Treeview(TableMargin, columns=('分析項分類','分析項','單位','每100克含量','名稱'), height=400, selectmode="extended",
                        yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set)
    scrollbary.config(command=tree.yview)
    scrollbary.pack(side=RIGHT, fill=Y)
    scrollbarx.config(command=tree.xview)
    scrollbarx.pack(side=BOTTOM, fill=X)
    tree.heading('分析項分類', text='分析項分類', anchor=W)
    tree.heading('分析項', text='分析項', anchor=W)
    tree.heading('單位', text='單位', anchor=W)
    tree.heading('每100克含量', text='每100克含量', anchor=W)
    tree.heading('名稱', text='名稱', anchor=W)
    tree.column('#0', stretch=NO, minwidth=0, width=0)
    tree.column('#1', stretch=NO, minwidth=0, width=90) #調整表頭欄位尺寸
    tree.column('#2', stretch=NO, minwidth=0, width=130)
    tree.column('#3', stretch=NO, minwidth=0, width=50)
    tree.column('#4', stretch=NO, minwidth=0, width=100)
    tree.column('#5', stretch=NO, minwidth=0, width=160)
    tree.pack()
    
   #http://passionfruittaiwan.blogspot.com/2019/01/python.html

    # 開啟中文營養資訊csv檔
    with open('D:/nutri_result.csv', 'r',encoding='utf-8') as f:
        # 讀取中文營養資訊csv檔
      reader = csv.DictReader(f, delimiter=',')
    #reader = pd.read_csv(r'\Users\wenruiling\Downloads\nutri_result.csv', 'r',encoding='utf-8')
    # 以迴圈輸出指定欄位
      for row in reader:          
        tag = row['分析項分類']
        nutri = row['分析項']
        unit = row['單位']
        has = row['每100克含量']
        name = row['名稱']
        # predict_name：欲篩選的蔬果名稱    
        # 比對predict_name和中文營養資訊檔案，將符合的資料插入表格中呈現
        if target in row['名稱']: 
            tree.insert("", 0, values=(tag,nutri,unit,has,name))

In [ ]:
# 創建視窗
window = tk.Tk()

# 設定視窗標題
window.title("Fruit ") 

# 視窗大小設定及背景顏色 
window.geometry("600x600") 
window.configure(background='white')

# 設定不讓使用者調整視窗大小
window.resizable(0,0) 

# 新增4個按鈕
# 1.選擇照片顯示於視窗中 
btn1 = Button(window, text ='Open image', command = open_img)
btn1.pack()
# 2.辨識水果名稱
btn2 = Button(window, text="Identify", command=identify) 
btn2.pack()
# 3.顯示得到的水果營養價值資料(英文)
btn3 = Button(window, text="Print Information", command=printSomething) 
btn3.pack()
# 4.顯示得到的水果營養價值資料(中文表格)
btn4 = Button(window, text="Nutrition Table", command=printNutriTable) 
btn4.pack()

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-28-c462707e2344>", line 19, in identify
    file_path = x
NameError: name 'x' is not defined
